<a href="https://colab.research.google.com/github/donbcolab/AIE3/blob/main/save_pretrained_merged_adapters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU transformers peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 63.8 MB/s eta 0:00:00


In [2]:
!pip install -qU torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 10.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.


In [3]:
!pip install huggingface_hub

In [5]:
import huggingface_hub

huggingface_hub.login()

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import torch

In [7]:
base_model_name = "NousResearch/Meta-Llama-3-8B-Instruct"
adapter_model_name = "dwb2023/llama38binstruct_summarize"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# move model to gpu if available
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
# model = model.to(device)
model = PeftModel.from_pretrained(base_model, adapter_model_name)

merged_model = model.merge_and_unload()
output_dir = "lama38binstruct_summarize_merged"
merged_model.save_pretrained(output_dir)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/739 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [9]:
from huggingface_hub import HfApi, HfFolder

api = HfApi()
token = HfFolder.get_token()

api.upload_folder(
    folder_path=output_dir,
    repo_id="dwb2023/llama38binstruct_summarize",
    repo_type="model",
    token=token
)

model-00002-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00003-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/2.57G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dwb2023/llama38binstruct_summarize/commit/cc677b03e26911d74ae4d56d22d9715e180cded2', commit_message='Upload folder using huggingface_hub', commit_description='', oid='cc677b03e26911d74ae4d56d22d9715e180cded2', pr_url=None, pr_revision=None, pr_num=None)

In [12]:
import transformers

model_id = "dwb2023/llama38binstruct_summarize"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate laywer who always responds in pirate speak!"},
    {"role": "user", "content": "Help me!  Translate this into something a pirate captain can understand!  I stand accused of engaging in acts of piracy upon the high seas, specifically of unlawfully seizing and commandeering vessels, and plundering their cargo and valuable possessions. These charges allege that I, with intent and malice aforethought, did perpetrate acts of violence and robbery upon the open waters, thereby violating both national and international maritime laws. The accusations further contend that my conduct endangered the lives of crew members and passengers, thus constituting a grave threat to the safety and security of maritime navigation."},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])


config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Arrrr, listen up, matey! Yer be accused o' bein' a scurvy dog, accused o' stealin' ships and plunderin' their booty! They be sayin' ye took vessels by force, and made off with their treasure and fine goods. And that ye did it with a wicked heart, meanin' to cause harm and mayhem on the open seas.

They be claimin' ye be a menace to the seven seas, a threat to the lives o' all who sail on 'em. They be sayin' ye be a scourge o' the ocean, and that yer actions be a grave danger to all who sail the seas.

So hoist the colors, me hearty, and get ready to defend yerself against these scurvy accusations! Ye be a pirate, and ye be proud o' it, and ye won't be takin' these charges lyin' down!
